## 01.- Apriori

### a) Importación de las librerías

In [5]:
#!pip install apyori

Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=b0c375b4396ffc45437c7089a5fd3637e83dff4c3d6d10af9d02d3f3476a5aab
  Stored in directory: c:\users\jose.lozas\appdata\local\pip\cache\wheels\32\2a\54\10c595515f385f3726642b10c60bf788029e8f3a1323e3913a
Successfully built apyori


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### b) Preprocesamiento de datos

In [7]:
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None)
transactions = []
for i in range(0, 7501):
  transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])

## Entrenamiento del modelo Apriori en el conjunto de datos

In [8]:
from apyori import apriori
rules = apriori(transactions = transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 2)

### c) Visualización de los resultados

### d) Visualización de los primeros resultados procedentes directamente de la salida de la función apriori

In [9]:
results = list(rules)

In [10]:
results

[RelationRecord(items=frozenset({'light cream', 'chicken'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'mushroom cream sauce', 'escalope'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'honey', 'fromage blanc'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0

### e) Poner los resultados bien organizados en un Pandas DataFrame

In [11]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

### f) Mostrar los resultados no ordenados

In [12]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843951
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
2,pasta,escalope,0.005866,0.372881,4.700812
3,fromage blanc,honey,0.003333,0.245098,5.164271
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
6,light cream,olive oil,0.003200,0.205128,3.114710
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
8,pasta,shrimp,0.005066,0.322034,4.506672


### g) Mostrar los resultados ordenados por ascensores descendentes

In [13]:
resultsinDataFrame.nlargest(n = 10, columns = 'Lift')

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.164271
0,light cream,chicken,0.004533,0.290598,4.843951
2,pasta,escalope,0.005866,0.372881,4.700812
8,pasta,shrimp,0.005066,0.322034,4.506672
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
6,light cream,olive oil,0.003200,0.205128,3.114710


El algoritmo Apriori es una técnica utilizada en la minería de datos y la búsqueda de patrones de asociación para descubrir relaciones frecuentes entre conjuntos de elementos en una base de datos. Se utiliza principalmente en la detección de patrones de compra en datos de transacciones. A continuación, proporcionaré una descripción detallada del algoritmo Apriori con enfoque matemático:

### 1. Definiciones Básicas
- $I$: Conjunto de elementos o ítems (por ejemplo, productos en una tienda en línea).
- $D$: Conjunto de transacciones, donde cada transacción $T$ es un subconjunto de $I$.
- $S$: Conjunto de elementos frecuentes (conjuntos de elementos que aparecen con suficiente frecuencia en $D$).

### 2. Concepto de Soporte
El soporte de un conjunto de elementos $X$, denotado como $\text{support}(X)$, se define como la proporción de transacciones en $D$ que contienen $X$. Matemáticamente:

$$
\text{support}(X) = \frac{\text{Número de transacciones en } D \text{ que contienen } X}{|D|}
$$

Donde \(|D|\) es el número total de transacciones en $D$.

### 3. Algoritmo Apriori
El algoritmo Apriori busca encontrar todos los conjuntos de elementos frecuentes ($S$) en $D$ mediante el uso de un proceso iterativo. La idea central es que si un conjunto de elementos $X$ es frecuente, entonces todos sus subconjuntos también deben ser frecuentes. El algoritmo sigue estos pasos:

#### a. Generación de Conjuntos de Tamaño 1
Inicia el proceso considerando cada elemento individual de $I$ como un conjunto de tamaño 1 y calcula su soporte. Los conjuntos de elementos individuales con un soporte igual o mayor que un umbral de soporte predefinido (generalmente especificado por el usuario) se consideran frecuentes y se agregan a $S$.

#### b. Generación de Conjuntos de Tamaño $k$
Luego, se generan conjuntos de elementos de tamaño $k$ a partir de los conjuntos de elementos frecuentes de tamaño $k-1$ encontrados en la iteración anterior. Estos nuevos conjuntos se prueban en $D$ para calcular su soporte. Los conjuntos de elementos frecuentes de tamaño $k$ se agregan a $S$.

#### c. Iteración
El proceso se repite iterativamente aumentando el tamaño de los conjuntos de elementos ($k$) hasta que no se pueden encontrar más conjuntos de elementos frecuentes.

### 4. Regla de Asociación
El algoritmo Apriori también se utiliza para generar reglas de asociación basadas en los conjuntos de elementos frecuentes. Una regla de asociación tiene la forma $X \rightarrow Y$, donde $X$ e $Y$ son conjuntos de elementos, y se interpreta como "si un cliente compra $X$, entonces también compra $Y$". La confianza de una regla se define como:

$$
\text{confianza}(X \rightarrow Y) = \frac{\text{support}(X \cup Y)}{\text{support}(X)}
$$

La confianza mide la probabilidad de que $Y$ se compre cuando $X$ ya está en el carrito de compras. Las reglas de asociación con una confianza superior a un umbral de confianza predefinido son consideradas válidas y útiles.

### 5. Ventajas y Desventajas
- **Ventajas:**
  - Permite descubrir patrones de compra frecuentes en datos de transacciones.
  - Ayuda en la toma de decisiones de marketing y recomendaciones de productos.
  - Puede utilizarse en una variedad de aplicaciones, como análisis de cesta de la compra y recomendación de contenido.
- **Desventajas:**
  - Puede ser computacionalmente costoso en grandes conjuntos de datos.
  - Requiere ajustar los umbrales de soporte y confianza de manera adecuada.
  - No tiene en cuenta la temporalidad de las transacciones.

### 6. Ejemplo Práctico
Supongamos que tienes un conjunto de datos que registra las transacciones de compra de clientes en una tienda en línea. Utilizando el algoritmo Apriori, puedes descubrir conjuntos de productos que a menudo se compran juntos, lo que te permite crear estrategias de ventas cruzadas y recomendaciones personalizadas.

En resumen, el algoritmo Apriori es una técnica que se utiliza para encontrar conjuntos de elementos frecuentes en datos de transacciones y generar reglas de asociación útiles. Ayuda a comprender los patrones de compra de los clientes y tomar decisiones informadas en marketing y recomendación de productos.